In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 12:50:00--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-27 12:50:00 (39.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch
import json

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
index.fit(documents)

In [6]:
q = 'the course has already started, can I still enroll?'

In [8]:
from openai import OpenAI

In [9]:

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [10]:
response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It is best to reach out to the course instructor or the institution offering the course to inquire about late enrollment options. Some courses may allow for late enrollment with permission from the instructor or may have a grace period for late registration. It ultimately depends on the specific course and institution's policies."

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
query = 'I just discovered the course. Can I still join it?'
rag(query)

'To run Kafka, you can follow these instructions based on the type of Kafka implementation you are working with:\n\n### Java Kafka\nIf you are running a Java Kafka producer, consumer, or kstreams, execute the following command in the project directory:\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\nEnsure you replace `<jar_name>` with the actual name of your JAR file.\n\n### Python Kafka\nIf you are running Kafka with Python and encountering issues like "Module ‘kafka’ not found," you should create a virtual environment and install the necessary packages:\n1. Create and activate a virtual environment:\n    ```sh\n    python -m venv env\n    source env/bin/activate  # For MacOS/Linux\n    ```\n\n    For Windows:\n    ```sh\n    env\\Scripts\\activate\n    ```\n\n2. Install the required packages:\n    ```sh\n    pip install -r ../requirements.txt\n    ```\n\n3. To activate the virtual environment in subsequent uses:\n    ```s

In [26]:
print(_)


"Yes, even if you don't register, you're still eligible to submit the homework assignments. However, please note that there will be deadlines for turning in the final projects, so it’s important not to leave everything until the last minute."